In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv('Diabetes.csv')
df.sample()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
528,0,117,66,31,188,30.8,0.493,22,Negative


In [6]:
# Encoding

df['Outcome']=df['Outcome'].astype('category')
df['Outcome']=df['Outcome'].cat.codes
df['Outcome']=df['Outcome'].astype('int')

In [7]:
x=df.drop('Outcome',axis=1)
y=df.Outcome

In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [9]:
# Feature Scaling

scale=MinMaxScaler()
x_train=scale.fit_transform(x_train)
x_test=scale.transform(x_test)

In [11]:
x_train.shape

(576, 8)

# HYPERPARAMETER TUNING

In [10]:
# Select the number of nodes/neurons in input,hidden
# Select the number of hidden layers
# Select the activation function for input,hidden and output
# Select the optimizer for the neural network

In [19]:
def build_model(hp):
  optimizers=hp.Choice('optimizer',['adam','sgd','rmsprop','adadelta'])
  num_layers=hp.Int('num_layers',min_value=1,max_value=20)

  model=Sequential()

  # INPUT LAYER:
  model.add(Dense(units=hp.Int('input_layer_node',min_value=2,max_value=128,step=2),activation=hp.Choice('input_layer_function',['sigmoid','relu','tanh','step']),input_dim=8))

  # HIDDEN LAYERS:
  for i in range(num_layers):
    model.add(Dense(units=hp.Int('hidden_layer_node'+str(i),min_value=2,max_value=128,step=2),activation=hp.Choice('hidden_layer_function'+str(i),['sigmoid','relu','tanh','step'])))

  # OUTPUT LAYER:
  model.add(Dense(units=1,activation=hp.Choice('output_layer_function',['sigmoid','relu','tanh','step'])))

  # COMPILING THE NEURAL NETWORK

  model.compile(optimizer=optimizers,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [14]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [15]:
import keras_tuner as kt

In [22]:
tuner=kt.RandomSearch(build_model,
                objective='val_accuracy',
                max_trials=3,
                directory='directory1',
                project_name='Final_project1')

In [23]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.625

Best val_accuracy So Far: 0.625
Total elapsed time: 00h 00m 09s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop',
 'num_layers': 5,
 'input_layer_node': 128,
 'input_layer_function': 'sigmoid',
 'hidden_layer_node0': 108,
 'hidden_layer_function0': 'tanh',
 'output_layer_function': 'tanh',
 'hidden_layer_node1': 2,
 'hidden_layer_function1': 'sigmoid',
 'hidden_layer_node2': 2,
 'hidden_layer_function2': 'sigmoid',
 'hidden_layer_node3': 2,
 'hidden_layer_function3': 'sigmoid',
 'hidden_layer_node4': 2,
 'hidden_layer_function4': 'sigmoid'}

In [25]:
model=tuner.get_best_models()[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Hyperparameter tuning ends

In [26]:
model.fit(x_train,y_train,epochs=100,initial_epoch=5,validation_data=(x_test,y_test),verbose=0)

In [28]:
train_loss,train_accuracy=model.evaluate(x_train,y_train)
test_loss,test_accuracy=model.evaluate(x_test,y_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6409 - loss: 0.6536 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6086 - loss: 0.6749 


In [29]:
print(train_accuracy)
print(test_accuracy)

0.6597222089767456
0.625


# CROSS VALIDATION

In [30]:
pip install scikeras

In [32]:
from scikeras.wrappers import KerasClassifier
mlcls=KerasClassifier(model,epochs=100,verbose=0)

In [33]:
from sklearn.model_selection import cross_val_score,KFold

accuracy=cross_val_score(mlcls,x_train,y_train,cv=KFold(3),scoring='accuracy').mean()

print('Mean accuracy =',accuracy)

Mean accuracy = 0.6597222222222223
